In [1]:
#recognize GPU
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-12-03 12:42:46.593380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-03 12:42:48.165339: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/lucas/miniconda3/lib/
2022-12-03 12:42:48.165457: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/lucas/miniconda3/lib/
2022-12-03 12:42:48.165468: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
#Download dataset (do only once)
#!ls -lha kaggle.json
#%pip install -q kaggle
#!mkdir -p ~/.kaggle
#!cp kaggle.json ~/.kaggle/
#!chmod 600 /root/.kaggle/kaggle.json
#!kaggle datasets download -d datasnaek/chess

In [3]:
#unzip
#!unzip chess.zip

In [4]:
#format data
text_file = "games.csv"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
movesarr = []

for line in lines:
    moves = line.split(",")
    movesarr.append(moves)

In [5]:
#Look at random game
import random

movesarr[random.randint(0, len(movesarr) -1)][12]

'e4 e5 Nf3 Nc6 Nc3 Nf6 Bc4 Bb4 Ng5 d5 exd5 Nd4 d6 Ne6 dxc7 Qxc7 Bxe6 fxe6 O-O O-O Nce4 Nxe4 Nxe4 Qc6 Re1 Rf4 Ng3 Rg4 c3 Bc5 Qxg4 b5 Rxe5 Bb7 Qxe6+ Qxe6 Rxe6 Rf8 d4 Bb6 b3 b4 cxb4 Bxd4 Rb1 Bxf2+ Kh1 Bd5 Re5 Ba8 Be3 Rf3 Bxf2 Rxf2 Re8+ Rf8 Rxf8+ Kxf8 Rc1 Ke7 Rc7+ Kd6 Rxa7'

In [6]:
#Filter out too low elo games
moveseq = []

for i in range(len(movesarr) -1): #Minus one because not looking at header in start
    if int(movesarr[i +1][9]) > 1750: #Do plus one because don't look at header
        moveseq.append(movesarr[i][12])
        
len(moveseq)

5746

In [7]:
#Make function to retrieve game sequence array easily
def getgame(num, seq):
    if not seq:
        return moveseq[num]
    else:
        return moveseq[num].split(" ")

In [8]:
def listToString(s):

    # initialize an empty string
    str1 = ""

    # traverse in the string
    for ele in s:
        if ele == s[-1]:
            str1 += f"{ele}"
        else:
            str1 += f"{ele} "
 
    # return string
    return str1

In [9]:
import random

segmented = []
for p in range(5):
    for i in range(len(getgame(p, True))):
        if i +1 != len(getgame(p, True)):
            segmented.append((f"{listToString(getgame(0, True)[:i +1])}", f"[start] {listToString(getgame(0, True)[i+1:i +2])} [end]"))

In [10]:
import random

segmented = []

for p in range(len(moveseq)):
    i = random.randrange(0, len(getgame(p, True)))
    segmented.append((f"{listToString(getgame(0, True)[:i +1])}", f"[start] {listToString(getgame(0, True)[i+1:i +2])} [end]"))

In [11]:
segmented[11]

('d4 d5 Nc3 Nf6 Bf4', '[start] Bf5 [end]')

In [12]:
#Partition data
import random
random.shuffle(segmented)

num_val_samples = int(0.15 * len(segmented))
num_train_samples = len(segmented) - 2*num_val_samples
train_samp = segmented[:num_train_samples]
val_samp = segmented[num_train_samples:num_train_samples + num_val_samples]
test_samp = segmented[num_train_samples + num_val_samples:]

## of samples for each category
print(num_val_samples)
print(num_train_samples)
print(len(test_samp))

861
4024
861


In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

def custom_standardization(input_string):
    return tf.strings.lower(input_string)


vocab_size = 15000
sequence_length = 20


source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize = custom_standardization
)
train_start_texts = [pair[0] for pair in train_samp]
train_end_texts = [pair[1] for pair in train_samp]
source_vectorization.adapt(train_start_texts)
target_vectorization.adapt(train_end_texts)

2022-12-03 12:42:50.089386: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-03 12:42:52.207922: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 106 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1
2022-12-03 12:42:52.253940: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 106.56M (111738880 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-12-03 12:42:52.254559: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:735] failed to allocate 95.91M (100564992 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


In [14]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_samp)
val_ds = make_dataset(val_samp)

In [15]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


2022-12-03 12:42:55.717554: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [16]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

In [18]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

2022-12-03 12:43:29.141586: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 14.65MiB (rounded to 15360000)requested by op StatelessRandomUniformV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-12-03 12:43:29.141627: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2022-12-03 12:43:29.141648: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 20, Chunks in use: 20. 5.0KiB allocated for chunks. 5.0KiB in use in bin. 141B client-requested in use in bin.
2022-12-03 12:43:29.141667: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 2, Chunks in use: 2. 1.2KiB allocated for chunks. 1.2KiB in use in bin. 1.0KiB client-requested in use in bin.
2022-12-03 12:43:29.141684: I tensorflow/tsl/fr

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[15000,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one.keras",
                                    save_best_only=True)
]

seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],)
seq2seq_rnn.fit(train_ds, epochs=2, validation_data=val_ds, callbacks=callbacks)

Epoch 1/2


2022-12-03 12:40:42.709925: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 61440000 exceeds 10% of free system memory.
2022-12-03 12:40:49.257005: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 61440000 exceeds 10% of free system memory.


 2/63 [..............................] - ETA: 4:43 - loss: 9.5956 - accuracy: 0.3022     

KeyboardInterrupt: 